# Debug Create Movie Pipeline (Low Memory)

This notebook breaks down the optimized `create_movie.py` pipeline.
Pipeline: Load (Per Channel) -> Stats -> BG Sub -> Norm & 8-bit -> Slab MIPs (2D) -> Blend Mips -> Assemble.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from types import SimpleNamespace
from pathlib import Path

if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

import create_movie

In [6]:
# 1. Configuration
args = SimpleNamespace(
    input_dir="/clusterfs/vast/abcabc/Korra_Foundation/20250805_mem-histone/fish1_24hpf/Chromatic_Shift_Corrected/Unmixed/matlab_stitch_no_xcorr/DSR/Cropped/",          # Change this to your path
    output="debug_movie.mp4",
    slab_size=10,
    num_slabs=None,
    voxel_size=[0.108, 0.108, 0.108],
    contrast_percentiles=[1, 99],
    background_offset=0.0,
    gamma=1.0,
    mock=False                # Set False for real data
)

In [7]:
# 2. Discovery
timeline = []
if args.mock:
    print("Generating MOCK timeline...")
    for t in range(0, 500, 100):
        timeline.append({
            'time_msec': t,
            'channels': [
                {'cam': 'CamA', 'ch': 'ch0', 'full_path': 'mock_path_A'},
                {'cam': 'CamB', 'ch': 'ch1', 'full_path': 'mock_path_B'}
            ]
        })
else:
    print(f"Scanning {args.input_dir}...")
    timeline = create_movie.discover_and_group_files(args.input_dir)

print(f"Found {len(timeline)} timepoints.")
bleaching_history = [[] for _ in range(len(timeline[0]['channels']))] if timeline else []

Scanning /clusterfs/vast/abcabc/Korra_Foundation/20250805_mem-histone/fish1_24hpf/Chromatic_Shift_Corrected/Unmixed/matlab_stitch_no_xcorr/DSR/Cropped/...
Found 80 timepoints.


In [8]:
# 3. Select Timepoint
t_idx = 0
if t_idx < len(timeline):
    item = timeline[t_idx]
    print(f"Processing {item['time_msec']} ms...")

Processing 79267094 ms...


## 4. Load & Process Per Channel
Loads, subtracts background, normalizes to 8-bit, and immediately computes 2D slabs.

In [ ]:
processed_volumes = []
raw_sums = []
channel_slabs_list = []

for f_meta in item['channels']:
    try:
        # Load
        vol = create_movie.load_zarr_data(f_meta['full_path'], mock=args.mock)
        
        # BG Sub (float)
        vol_bg = vol.astype(np.float32) - args.background_offset
        vol_bg[vol_bg < 0] = 0
        
        # Stats (on raw floating point data)
        raw_sums.append(np.sum(vol_bg))
        
        # Norm -> 8-bit
        vol_norm = create_movie.normalize_volume(vol_bg, args.contrast_percentiles[0], args.contrast_percentiles[1], args.gamma)
        processed_volumes.append(vol_norm)
        
        # Compute Slabs (Single Channel)
        slabs = create_movie.create_slabs_mip(vol_norm, slab_size=args.slab_size, num_slabs=args.num_slabs)
        channel_slabs_list.append(slabs)
        
        print(f"Processed {f_meta['cam']}_{f_meta['ch']}: {vol.shape} -> 8-bit -> {len(slabs)} slabs")
        
    except Exception as e:
        print(f"Error: {e}")

# Mock history update
if len(bleaching_history[0]) <= t_idx:
    for i, val in enumerate(raw_sums):
        bleaching_history[i] = [val] * (t_idx + 1)

# Visual Check: First slab of first channel
if channel_slabs_list:
    plt.figure(figsize=(6,6))
    plt.imshow(channel_slabs_list[0][0], cmap='gray')
    plt.title("Ch0 - Slab 0 MIP")
    plt.show()

## 5. Blend & Arrange MIPs
Combine per-channel slabs into RGB MIPs.

In [ ]:
final_rgb_mips = []
if channel_slabs_list:
    num_generated_slabs = len(channel_slabs_list[0])
    for s_i in range(num_generated_slabs):
        # Gather slabs for this index
        slabs_to_blend = []
        for ch_slabs in channel_slabs_list:
            if s_i < len(ch_slabs):
                slabs_to_blend.append(ch_slabs[s_i])
            else:
                 slabs_to_blend.append(None)
        
        rgb_slab = create_movie.blend_channel_mips(slabs_to_blend, item['channels'])
        final_rgb_mips.append(rgb_slab)

mips_img = create_movie.arrange_mips(final_rgb_mips, args.voxel_size)

plt.figure(figsize=(12, 12))
plt.imshow(mips_img)
plt.title("Arranged RGB MIPs")
plt.axis('off')
plt.show()

## 6. FFT Analysis
Using the kept 8-bit volumes (summed).

In [ ]:
sum_vol = np.sum(np.array(processed_volumes), axis=0)
fft_xy, fft_xz, fft_yz = create_movie.compute_fft_planes(sum_vol)
fft_combined = create_movie.combine_ppt_planes(fft_xy, fft_xz, fft_yz)

plt.figure(figsize=(10, 4))
plt.imshow(fft_combined, cmap='inferno')
plt.title("FFT Principal Planes")
plt.axis('off')
plt.show()

## 7. Bleaching Stats

In [ ]:
plt.figure(figsize=(6, 4))
times = [x['time_msec'] for x in timeline[:t_idx+1]]
for c_i, hist in enumerate(bleaching_history):
    if len(hist) > 0:
        norm_vals = np.array(hist) / hist[0] if hist[0] > 0 else np.array(hist)
        plt.plot(times, norm_vals[:len(times)], label=f"Ch{c_i}")
plt.legend()
plt.title("Bleaching (Integrated Intensity)")
plt.show()

## 8. Final Frame Assembly

In [ ]:
fig = plt.figure(figsize=(18, 10), constrained_layout=True)
gs = fig.add_gridspec(2, 3)

ax_mips = fig.add_subplot(gs[:, 0:2])
ax_fft = fig.add_subplot(gs[0, 2])
ax_bleach = fig.add_subplot(gs[1, 2])

ax_mips.imshow(mips_img)
ax_mips.axis('off')
ax_mips.set_title("MIPs")

ax_fft.imshow(fft_combined, cmap='inferno')
ax_fft.axis('off')

for c_i, hist in enumerate(bleaching_history):
    if len(hist) > 0:
        norm_vals = np.array(hist) / hist[0] if hist[0] > 0 else np.array(hist)
        ax_bleach.plot(times, norm_vals[:len(times)], label=f"Ch{c_i}")
ax_bleach.legend()

plt.show()